这段脚本展示了如何使用LlamaIndex库中的**Ollama模型**进行文本生成和聊天功能，并且还演示了如何将嵌入式模型和索引组件集成到一个完整的查询系统中。具体来说，它包括以下几个部分：

- 文本完成：使用Ollama模型生成给定前缀的后续内容。
- 聊天对话：模拟与AI助手进行简单的问答交互。
- 构建和查询索引：通过加载文档创建关键词表索引，并基于此索引回答问题。
- 嵌入式模型集成：使用嵌入式模型来表示文档，以便在查询时可以更高效地检索相关信息。

In [1]:
from llama_index.llms.ollama import Ollama

base_url='http://localhost:11434'
model=Ollama(model="qwen2.5:latest", request_timeout=360.0,base_url=base_url)

In [2]:
# 功能1:文本完成
resp=model.complete('LlamaIndex是')
print(resp)

LlamaIndex 是一个由 GitHub 用户 thunlp 开发的开源库，它主要用于构建交互式的语言模型驱动的知识索引系统。这个工具可以帮助用户将大量的文本数据高效地转化为结构化的知识图谱，并提供相应的查询接口来实现高效检索。

LlamaIndex 通过使用不同的数据结构（如树形结构、哈希表等）以及智能的分词和索引技术，使得大规模的文本可以被快速查找和定位。这对于构建复杂的问答系统、文档搜索引擎或者任何需要处理大量文本信息的应用程序都非常有用。

此外，它还可以与流行的自然语言处理工具链集成，比如 LlamaIndex 可以结合使用如 LangChain（一个用于构建强大而灵活的 AI 应用程序的 Python 框架）来增强其功能。LlamaIndex 提供了多种类型的索引节点，可以适应不同的应用场景需求。

总之，LlamaIndex 是一个非常有用的工具库，它简化了知识图谱的创建和查询过程，适用于需要处理大规模文本数据的各种场景。


In [3]:
# 功能2: 聊天
from llama_index.core.llms import ChatMessage

messages=[
    ChatMessage(
        role='system',
        content='你是一位专门回答LlamaIndex问题的客服，叫小Index'
    ),
    ChatMessage(
        role='user',
        content='你叫什么名字？'
    )
]

resp=model.chat(messages)
print(resp)

assistant: 我是小Index，很高兴为您服务！有什么关于LlamaIndex的问题都可以问我哦。


In [5]:
# 使用：作为组件嵌入到其他模块（索引、检索器、查询引擎、代理）
from llama_index.core import KeywordTableIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama

documents = SimpleDirectoryReader("../../data/三国演义白话文/").load_data(show_progress=True)

# 自定义模型及参数
llm = Ollama(
    temperature=0.1, 
    model="qwen2.5:latest",
    base_url=base_url)

# build index
index = KeywordTableIndex.from_documents(documents, llm=llm,show_progress=True)

# get response from query
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("赵子龙什么时候出场的？")
print(response)

Loading files: 100%|██████████| 41/41 [00:00<00:00, 429.14file/s]
c:\Users\wushaogui\miniconda3\envs\langchian\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting keywords from nodes: 100%|██████████| 127/127 [03:01<00:00,  1.43s/it]


赵子龙是在袁绍与公孙瓒交战时危急时刻出场的，他从山坡边闪出救了公孙瓒。


除了llms外，嵌入模型也可以作为单独使用

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.ollama import OllamaEmbedding

documents = SimpleDirectoryReader("../../data/水浒传白话文/",recursive=True).load_data(show_progress=True)

# bge-base embedding model
embed_model = OllamaEmbedding(model_name="quentinz/bge-large-zh-v1.5:latest",base_url=base_url)

index = VectorStoreIndex.from_documents(documents, embed_model=embed_model,show_progress=True)

Generating embeddings: 100%|██████████| 308/308 [00:15<00:00, 19.74it/s]


In [10]:
# get response from query
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("风雪山神庙发生了什么事？")
print(response)

风雪山神庙时，林冲正准备在山神庙过夜。突然，草料场起火了，火光照亮了山神庙。原来，陆谦、富安与差拨三人为了除掉林冲，在草料场放了火。林冲听到外面的对话，确认是这三个人所为，并且从庙中杀出，但那三人吓得逃走了。
